In [1]:
!pip install accelerate gTTS gradio transformers

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from gtts import gTTS
import torch
import gradio as gr

In [3]:

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
device

'cpu'

In [5]:
language_model_name = "Qwen/Qwen2-1.5B-Instruct"
language_model = AutoModelForCausalLM.from_pretrained(
    language_model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(language_model_name)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Some parameters are on the meta device because they were offloaded to the disk.


In [6]:
def process_input(input_text, action):
    if action == "Translate to English":
        prompt = f"Please translate the following text into English：{input_text}"
        lang = "en"
    elif action == "Translate to German":
        prompt = f"Please translate the following text into German：{input_text}"
        lang = "de"
    elif action == "Translate to Hindi":
        prompt = f"Please translate the following text into Hindi：{input_text}"
        lang = "hi"
    else:
        prompt = input_text
        lang = "en"

    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = language_model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    output_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return output_text, lang

In [7]:

def text_to_speech(text, lang):
    tts = gTTS(text=text, lang=lang)
    filename = "output_audio.mp3"
    tts.save(filename)
    return filename

In [8]:

def handle_interaction(input_text, action):
    output_text, lang = process_input(input_text, action)
    audio_filename = text_to_speech(output_text, lang)
    return output_text, audio_filename

In [9]:
action_options = ["Translate to English", "Translate to German", "Translate to Hindi", "Chat"]

In [10]:
iface = gr.Interface(
    fn=handle_interaction,
    inputs=[
        gr.Textbox(label="input text"),
        gr.Dropdown(action_options, label="select action")
    ],
    outputs=[
        gr.Textbox(label="output text"),
        gr.Audio(label="output audio")
    ],
    title="Translation and Chat App using AI",
    description="Translate input text or chat based on the selected action.",
    theme= "gradio/soft"
)

In [11]:

if __name__ == "__main__":
    iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://ecea4d27c9a6b8f9a2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/opt/anaconda3/lib/python3.12/site-packages/transformers/generation/utils.py:2105: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/transformers/generation/utils.py:2105: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpec